In [ ]:
from pfs.lam.archiverConf import getConf, extractData, colors
import numpy as np
from matplotlib.dates import num2date, date2num
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
def cooler_lift(T_tip, W_cooler, T_0=40.8022826, T_1=100., W_0=240., k=30.9095981, m=-0.3734864, n=0.07916974):
    return (T_tip-T_0)/(T_1-T_0)*k*((W_cooler/W_0)+m*(W_cooler/W_0)**2+n*(W_cooler/W_0)**3)

def cryoP(t, a=10.285, b=2637.006):
    return 10**(a - (b / t))

def cryoT(p, a=10.285, b=2637.006):
    return b / (a - np.log10(p))

In [ ]:
dbname='archiver_2015'

In [ ]:
start = '2018-12-20 14:30'
end = '2018-12-21 09:05'

In [ ]:
d = getConf(start, actors=['xcu_b1'], dbname=dbname)

In [ ]:
tables = ['xcu_b1__pressure', 'xcu_b1__coolertemps', 'xcu_b1__temps']

In [ ]:
df = extractData(tables, start, end, interpolate='linear',  dbname=dbname)

In [ ]:
df.columns

In [ ]:
df['lift'] = cooler_lift(df.tip, df.power)

In [ ]:
df['cryop'] = cryoP(df['Thermal Spreader Assy'])
df['cryot'] = cryoT(df['Pressure'])

In [ ]:
ax = df.set_index('date')[['cryop']].plot(figsize=(12, 10), grid=True, legend=True)
ax.set_yscale('log')

ax.semilogx(df.tai, df['Pressure'], label='pressure')
ax.legend()

In [ ]:
ax = df.set_index('date')[['cryot']].plot(figsize=(12, 10), grid=True, legend=True)

ax.plot(df.tai, df['tip'], label='tip')
ax.plot(df.tai, df['Thermal Spreader Assy'], label='Thermal Spreader')
ax.plot(df.tai, df['Detector Strap 1'], label='Detector')
ax.legend()